In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import mysql.connector
from tqdm.notebook import tqdm
import pandas as pd
import time

In [2]:
def connect_db():
    db = mysql.connector.connect(
        host="101.101.210.141",
        user="study",
        passwd="study!@#$%",
        database="seoul_welfare"
        # database="aihub_seoul_healthcare"
)
    return db

def mysql_query(database, query:str):
    database.reconnect()
    cur = database.cursor() 
    cur.execute(query)
    result = cur.fetchall()

    return result

In [3]:
db = connect_db()

In [4]:
eval_data_path = '../data/welfare/test.jsonl'
evalset = load_dataset("json", data_files = eval_data_path, split='train')
len(evalset)

136

In [5]:
evalset['input'][1]

'폐지한 민간 어린이집의 이름을 알려줘'

In [6]:
max_seq_length = 2048
dtype = torch.float16
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "FINwillson/llama-3-8B-welfare-sft-v1",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.642 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
model.config.to_dict()["torch_dtype"]

'float16'

In [8]:
prompt_1 = """### Task
Generate a SQL query to answer the following question:
`{}`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"{}"
An example of the SQL would be 'SELECT * FROM orders WHERE DATE(orderdate) = CURDATE() - INTERVAL 1 DAY AND status = 'COMPLETED'
 
### SQL
Given the database schema, here is the SQL query that answers `{}`:
[SQL]{}"""

In [9]:
test = prompt_1.format(
            f"{evalset[1]['input']}",
            f"{evalset[1]['context']}",
            f"{evalset[1]['input']}",
            "", 
        )

In [10]:
print(test)

### Task
Generate a SQL query to answer the following question:
`폐지한 민간 어린이집의 이름을 알려줘`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"CREATE TABLE child_care_info_y_s (  SIGUNNAME varchar(8) DEFAULT NULL,  STCODE bigint DEFAULT NULL,  CRNAME varchar(32) DEFAULT NULL,  CRTYPENAME varchar(11) DEFAULT NULL,  CRSTATUSNAME varchar(7) DEFAULT NULL,  ZIPCODE bigint DEFAULT NULL,  CRADDR varchar(52) DEFAULT NULL,  CRTELNO varchar(18) DEFAULT NULL,  CRHOME varchar(44) DEFAULT NULL,  NRTRROOMSIZE varchar(9) DEFAULT NULL,  PLGRDCO varchar(6) DEFAULT NULL,  CCTVINSTLCNT varchar(7) DEFAULT NULL,  CHCRTESCNT varchar(7) DEFAULT NULL,  CRCAPAT varchar(8) DEFAULT NULL,  CRCHCNT varchar(8) DEFAULT NULL,  LA bigint DEFAULT NULL,  LO bigint DEFAULT NULL,  CRCARGBNAME varchar(8) DEFAULT NULL,  CRCNFMDT varchar(19) DEFAULT NULL,  CRPAUSEBEGINDT varchar(200) DEFAULT NULL,  CRABLDT varchar(19) DEFAULT NULL,  DATASTDRDT varchar(19) DEFAULT NULL,  CRSPEC var

In [11]:
excute_result = []
query_cnt_result = []
include_query_result = []
cnt_query_result = []
inference_query = []
fail_check = []
FastLanguageModel.for_inference(model) 

In [12]:
for i in tqdm(range(len(evalset))):
    inputs = tokenizer(
    [
        prompt_1.format(
            f"{evalset[i]['input']}",
            f"{evalset[i]['context']}",
            f"{evalset[i]['input']}",
            "", 
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
    result = tokenizer.batch_decode(outputs)
    result = result[0].split("[SQL]")[-1].split("[/SQL]")[0].split("<|end_of_text|>")[0]
    torch.cuda.empty_cache()
    inference_query.append(result)
# print(f"index:{i}\ninference:{result}")

# try:
#     infer_query_result = mysql_query(db, result)
#     real_query_result = mysql_query(db, evalset['output'][i])
#     comp = any(item in real_query_result for item in infer_query_result)
#     excute_result.append(1)
#     cnt_query_result.append(len(infer_query_result))
#     if len(infer_query_result) == len(real_query_result):
#         query_cnt_result.append(1)
#     else:
#         query_cnt_result.append(0)
#         fail_check.append(i)
    
#     if comp == True:
#         include_query_result.append(1)
#     else:
#         include_query_result.append(0)
# except Exception as e:
#     excute_result.append(0)
#     cnt_query_result.append(0)
#     query_cnt_result.append(0)
#     print(i, e)
#     fail_check.append(i)

#     time.sleep(0.2)
    
# print(f"index:{i}\ninference:{result}\nLabel:{inference_query}")

  0%|          | 0/136 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [13]:
inference_query

[' SELECT AVG(CAST(Q_SUM AS SIGNED)) FROM global_job_counsel_qtrst WHERE YEAR(YYYY) = 2020\n',
 " SELECT CRNAME FROM child_care_info_y_s WHERE CRTYPENAME = '민간' AND CRSTATUSNAME = '폐지' AND CRNFMDT!= '0000-00-00 00:00:00';\n",
 ' SELECT * FROM subway_tour_info WHERE STATION = "건대입구역"\n \n### Result\nThe result is a table containing a single row with a single column containing a single value:\n```sql\nNO\nLINE\nLINE_NAME\nSTATION\nSTATION_IMAGE\nCOURSE1\nCOURSE2\nELEVATER\nELEVATER_TXT\nSTATION_IMAGE2\nUSEYN\nICON_PATH\nEXIT_INFO\nTELNO_INFO\nTIME_INFO1\nTIME_INFO2\nTIME_INFO3\nTIME_INFO4\n```\n',
 " SELECT * FROM odsn_building_info WHERE addr LIKE '마포%' AND addr LIKE '%경로당%'\n \n### Sample Output\nThe sample output for this query would be:\n|ID|NM|ADDR_OLD|ADDR|PERMISSION_DT|STATE|STOP_DT|SUSPENSION_START_DT|SUSPENSION_END_DT|RE_OPEN_DT|AREA|POST|ENTRNC_PSNCPA|CTLF_NUM|TOTAL_NUM|XCODE|YCODE|PERMISSION_NO|DETAIL_STAT_NM|\n|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|\n|01010

In [13]:
for idx, query in enumerate(tqdm(inference_query)):
    try:
        infer_query_result = mysql_query(db, query)
        real_query_result = mysql_query(db, evalset['output'][idx])
        comp = any(item in real_query_result for item in infer_query_result)
        excute_result.append(1)
        cnt_query_result.append(len(infer_query_result))
        if len(infer_query_result) == len(real_query_result):
            query_cnt_result.append(1)
        else:
            query_cnt_result.append(0)
            fail_check.append(idx)
            
        if comp == True:
            include_query_result.append(1)
        else:
            include_query_result.append(0)
    except Exception as e:
        excute_result.append(0)
        cnt_query_result.append(0)
        query_cnt_result.append(0)
        print(idx, e)
        fail_check.append(idx)

    time.sleep(0.2)

  0%|          | 0/136 [00:00<?, ?it/s]

12 1055 (42000): Expression #2 of SELECT list is not in GROUP BY clause and contains nonaggregated column 'seoul_welfare.GLOBAL_JOB_COUNSEL_QTRST.Q_SUM' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by
27 1054 (42S22): Unknown column 'MID_NM' in 'field list'
70 1054 (42S22): Unknown column 'STALL' in 'where clause'


In [14]:
print(f"쿼리 실행 확률 : {round(excute_result.count(1) / len(evalset), 2)}\n\
쿼리 결과 수 같을 확률 : {round(query_cnt_result.count(1) / len(evalset), 2)}")

쿼리 실행 확률 : 0.98
쿼리 결과 수 같을 확률 : 0.86


In [13]:
check_fail_idx = []
for idx, val in enumerate(query_cnt_result):
    if val == 0:
        check_fail_idx.append(idx)

In [15]:
len(check_fail_idx)

358

In [15]:
len(inference_query)

507

In [12]:
evalset

Dataset({
    features: ['input', 'accept', 'reject', 'context'],
    num_rows: 100
})

In [14]:
import pandas as pd
input = pd.Series(evalset['input'])
rejected = pd.Series(inference_query)
chosen = pd.Series(evalset['accept'])
context =  pd.Series(evalset['context'])

df = pd.concat([input, chosen, rejected, context], axis=1)
df.rename(columns={ 0: 'input', 1: 'accept', 2: 'reject', 3: 'context'}, inplace = True)
df.to_csv("../dpo/data/0514university_dpo_kor_data.csv")
df

,input,accept,reject,context
0,모든 모듈에 대해 각 학생이 등록한 총 모듈 수를 계산하고 학생과 등록한 총 모듈 ...,"SELECT id_student, COUNT(*) AS total_modules_r...","SELECT sr.id_student, COUNT(sr.code_module) AS...",CREATE TABLE studentregistration (code_module ...
1,지역명과 지역별 Pass 등급을 받은 학생들의 평균 학점,"SELECT region, AVG(studied_credits) AS avg_cre...","SELECT region, AVG(studied_credits) AS avg_cre...",CREATE TABLE studentinfo (code_module varchar(...
2,가장 많은 학점을 이수한 모듈의 학생번호 및 최종 성적을 조회,"SELECT sr.id_student, si.final_result FROM stu...","SELECT sr.id_student, si.final_result FROM stu...",CREATE TABLE studentinfo (code_module varchar(...
3,등록한 학생들 중 VLE 자료와 상호 작용한 학생들의 연령대별 평균 학점이 80보다...,"SELECT s.age_band, AVG(studied_credits) AS avg...","SELECT studentinfo.age_band, AVG(studentinfo.s...",CREATE TABLE studentinfo (code_module varchar(...
4,각 모듈 프레젠테이션에 등록한 학생 중 VLE 자료를 이용한 학생의 평균 상호 작용...,"SELECT sr.code_module, sr.code_presentation, A...","SELECT studentregistration.code_module, studen...",CREATE TABLE studentinfo (code_module varchar(...
...,...,...,...,...
393,각 지역의 지역명과 학생들의 평균 나이를 조회,"SELECT region, AVG(CAST(SUBSTRING(age_band, 1,...","SELECT region, AVG(age_band) AS avg_age FROM s...",CREATE TABLE studentinfo (code_module varchar(...
394,평균 클릭 수 보다 높은 클릭 수의 학생들의 평균 성적을 구해줘,SELECT AVG(studentassessment.score) AS average...,SELECT AVG(studentassessment.score) AS average...,CREATE TABLE studentassessment (id_assessment ...
395,평가 결과가 'Distinction'이고 모든 평가 유형에서 평균 점수가 80보다 ...,SELECT COUNT(*) AS distinction_count FROM ( SE...,SELECT COUNT(DISTINCT studentinfo.id_student) ...,CREATE TABLE studentassessment (id_assessment ...
396,각 모듈 프레젠테이션에서 가장 높은 점수를 받은 학생의 최종 결과를 식별하고 모듈 ...,"SELECT sr.code_module, sr.code_presentation, s...","SELECT sr.code_module, sr.code_presentation, s...",CREATE TABLE studentinfo (code_module varchar(...


In [9]:
i = 1
inputs = tokenizer(
[
    prompt_1.format(
        f"{evalset[i]['input']}",
        f"{evalset[i]['context']}",
        f"{evalset[i]['input']}",
        "", 
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)
result = result[0].split("[SQL]")[-1].split("[/SQL]")[0]

# infer_query_result = mysql_query(db, result)
# real_query_result = mysql_query(db, evalset['output'][i])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
from pprint import pprint
print(result)

SELECT c.code_module, AVG(c.module_presentation_length) as avg_presentation_length, v.activity_type, MAX(sr.date_registration) as max_date_registration FROM courses c JOIN vle v ON c.code_module = v.code_module AND c.code_presentation = v.code_presentation JOIN studentregistration sr ON c.code_module = sr.code_module AND c.code_presentation = sr.code_presentation GROUP BY c.code_module, v.activity_type;


In [13]:
evalset[i]['output']

'SELECT c.code_module, AVG(c.module_presentation_length) AS avg_presentation_length, v.activity_type, AVG(v.week_to - v.week_from) AS avg_activity_duration FROM courses c JOIN vle v ON c.code_module = v.code_module AND c.code_presentation = v.code_presentation JOIN studentRegistration r ON c.code_module = r.code_module AND c.code_presentation = r.code_presentation GROUP BY c.code_module, v.activity_type;'

### test

In [8]:
class EosTokenRewardLogitsProcessor:
  def __init__(self, eos_token_id: int, max_length: int):
    # `eos_token_id`와 `max_length`의 유효성을 검사하는 조건문입니다.
    if not isinstance(eos_token_id, int) or eos_token_id < 0:
        raise ValueError(f"`eos_token_id` has to be a positive integer, but is {eos_token_id}")

    if not isinstance(max_length, int) or max_length < 1:
        raise ValueError(f"`max_length` has to be an integer bigger than 1, but is {max_length}")

    # 생성자 메서드에서 `eos_token_id`와 `max_length`를 클래스 멤버 변수로 저장합니다.
    self.eos_token_id = eos_token_id
    self.max_length = max_length

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
    cur_len = input_ids.shape[-1]
    # 보상을 증가시키는 과정을 설명하는 주석입니다.
    # eos_token의 보상을 문장 길이에 따라 조절합니다.
    for cur_len in (max(0, int(self.max_length * 0.8)), self.max_length):
      ratio = cur_len / self.max_length
      num_tokens = scores.shape[1]  # 어휘 크기 (단어 집합의 크기)

      # eos_token이 아닌 다른 토큰의 점수를 조정합니다.
      # 선택된 토큰의 부호(sign)의 지수값을 취한 것으로, 음수 값은 0에 가깝게, 양수 값은 1에 가깝게 변환합니다.
      scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]] = \
        scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]] * ratio * 10 * torch.exp(-torch.sign(scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]]))

      # eos_token의 보상을 특정 상수로 설정합니다.
      scores[:, self.eos_token_id] = 1e2 * ratio

    # 조정된 점수가 저장된 scores를 반환합니다.
    return scores

In [10]:
from pprint import pprint
from googletrans import Translator

translator = Translator()

In [20]:
j = 10
korean_input = df['질문'][j]
label = df['답변'][j]
question = translator.translate(korean_input, dest='en').text
inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{question}", # input
            "", 
        )
    ], return_tensors = "pt").to("cuda")
# pprint(inputs)
outputs = model.generate(**inputs,
                         max_new_tokens = 300,
                        #  pad_token_id= tokenizer.eos_token_id,
                        #  eos_token_id= tokenizer.eos_token_id,
                         output_scores=True,
                        #  logits_processor =[EosTokenRewardLogitsProcessor(eos_token_id=tokenizer.eos_token_id, max_length=300)],
                         use_cache = True)
result = tokenizer.batch_decode(outputs)
result = result[0].split("### Response:")[-1].split(";")[0].split("<|end_of_text|>")[0].replace("\n", "") + ";"
torch.cuda.empty_cache()
# print(f"Inference: {result}\nLabel: {evalset['output'][j]}")
print(f"Inference: {result}\nLabel: {label}")
infer_query_result = mysql_query(db, result)
real_query_result = mysql_query(db, label)
comp = any(item in real_query_result for item in infer_query_result)
pritn(f"\ncomparison: {comp}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference: SELECT AVG(studied_credits) FROM studentInfo;
Label: SELECT AVG(studied_credits) as avg_credits
FROM studentInfo;


NameError: name 'mysql_query' is not defined